In [ ]:
%pip install pandas tqdm pyarrow

In [19]:
import pandas as pd
import re
from tqdm import tqdm

In [ ]:
!pip install -r requirements.txt

In [1]:
!wget -P files https://dumps.wikimedia.org/other/pageview_complete/2020/2020-01/pageviews-20200101-user.bz2

--2021-10-06 10:01:54--  https://dumps.wikimedia.org/other/pageview_complete/2020/2020-01/pageviews-20200101-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:1:208:80:154:7, 208.80.154.7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:1:208:80:154:7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 488445774 (466M) [application/octet-stream]
Saving to: ‘pageviews-20200101-user.bz2’

                      7%[>                   ]  35.60M  4.87MB/s    eta 97s    ^C


In [ ]:
!bzip2 -d files/pageviews-20200101-user.bz2

In [ ]:
!./fixcol.sh files/pageviews-20200101-user files/pageviews-20200101-user-fixed

In [20]:
filename = "files/pageviews-20200101-user-fixed"
year, month, day = filename.split("-")[1][:4], filename.split("-")[1][4:6], filename.split("-")[1][6:]
print(year, month, day)

2020 01 01


In [21]:
df = pd.read_csv(filename, sep=" ", names=["domain", "title", "access_type", "count", "summary"])

In [22]:
df.head(n=100)

,domain,title,access_type,count,summary
0,aa.wikibooks,-,desktop,2,A2
1,aa.wikibooks,Main_Page,desktop,83,A2B1C8E1G3H4I3J2K27L8M1O1P2Q2S3T2U4V4W3X2
2,aa.wikibooks,Main_Page,mobile-web,5,C1D1I1Q1S1
3,aa.wikibooks,Special:CreateAccount,desktop,17,B1C1G2I2J2L1P1Q1R1T2V1W1X1
4,aa.wikibooks,Special:UserLogin,desktop,20,B1C1G2I2J2K1L1M1P1Q1R1T2U1V1W1X1
...,...,...,...,...,...
95,ab.wikipedia,1389,desktop,1,J1
96,ab.wikipedia,1390,mobile-web,1,U1
97,ab.wikipedia,14,desktop,1,D1
98,ab.wikipedia,1400,desktop,1,C1


In [23]:
new_df = pd.DataFrame(columns=["domain", "title", "access_type", "summary", "year", "month", "day", "hour", "count"])

values = []
for idx, row in tqdm(df.iterrows()):
    raw_hour_count_pairs = re.findall(r"([A-X])(\d+)", row["summary"])
    hour_count_pairs = [{"hour": ord(enc[0])-ord('A'), "count": int(enc[1])} for enc in raw_hour_count_pairs]
    for hcp in hour_count_pairs:
        values.append({
            **row,
            **hcp,
            "year": year,
            "month": month,
            "day": day
        })
new_df = pd.DataFrame.from_dict(values)

13660393it [10:41, 21307.28it/s]


KeyboardInterrupt: 

In [ ]:
new_df.columns

In [26]:
new_df = pd.DataFrame.from_dict(values)
new_df.to_parquet("files/pageviews-20200101-user-fixed2.parquet", partition_cols=["year", "month", "day"])

In [29]:
r = pd.read_parquet("files/pageviews-20200101-user-fixed2.parquet")